In [ ]:
import os, warnings
warnings.filterwarnings('ignore')


In [ ]:
import gc
import numpy as np
import cv2, glob
from random import shuffle, randint
from tqdm import tqdm
import pydot
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization
from tensorflow.keras.layers import Activation, Conv2DTranspose, Conv2D, LeakyReLU
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from keras.layers.convolutional import _Conv
from keras.legacy import interfaces
from keras.engine import InputSpec
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

from keras import Input
from keras.layers import UpSampling2D,Concatenate
from IPython.display import SVG
from keras import Model
from keras.models import model_from_json
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


In [ ]:
image_files_anime = glob.glob('../input/anime-faces/data/data/*.png')
shuffle(image_files_anime)
image_files_train = glob.glob('../input/animeface/faces/faces/*.jpg')
image_files_face = glob.glob('../input/appa-real-face-cropped/final_files/final_files/*.jpg')
shuffle(image_files_face)

In [ ]:

x_i = []
c=0
for file in tqdm(image_files_train):
    image = cv2.imread(file)
    image = cv2.resize(image,(64,64))
    image = image / 127.5 -1
    x_i.append(image)
    c+=1
    if c==23000:
        break
gc.collect()
test = np.array(x_i[20000:])
test.shape
x_i = np.array(x_i[:20000])
x_i.shape

x_t = []
for file in tqdm(image_files_face):
    image = cv2.imread(file)
    image = cv2.resize(image,(64,64))
    image = image / 127.5 -1
    x_t.append(image)
x_t = np.array(x_t)
x_t.shape

In [ ]:
test.shape

In [ ]:
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (16, 16))
plt.setp(axes.flat, xticks = [], yticks = [])
for i, ax in enumerate(axes.flat):
    index = randint(0, 7500)
    ax.imshow(x_t[index][:,:,::-1],cmap='gray')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (16, 16))
plt.setp(axes.flat, xticks = [], yticks = [])
for i, ax in enumerate(axes.flat):
    index = randint(0, 7500)
    ax.imshow(x_i[index][:,:,::-1])
plt.show()

In [ ]:
class CycleGAN():
    def __init__(self, img_rows=64, img_cols=64, channels=3):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        
        self.img_shape = (img_rows, img_cols, channels)
        
        self.disc_patch = (4,4,1)
        self.residual_blocks = 6
        
        
        self.gf = 32
        self.df = 64
        
        self.lambda_cycle = 10.0
        
        self.lambda_id = 0.9 * self.lambda_cycle
        
        optimizer = Adam(0.0002, 0.5)
        
        self.d_a = self.build_discriminator('Discriminator_A')
        self.d_b = self.build_discriminator('Discriminator_B')

        self.d_a.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        self.d_b.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

        self.g_ab = self.build_generator('Generator_AB')
        self.g_ba = self.build_generator('Generator_BA')
        
        img_a = Input(shape=self.img_shape,name='Input_A')
        img_b = Input(shape=self.img_shape,name='Input_B')
        
        fake_b = self.g_ab(img_a)
        fake_a = self.g_ba(img_b)
        
        recon_a = self.g_ba(fake_b)
        recon_b = self.g_ab(fake_a)
        
        img_a_id = self.g_ba(img_a)
        img_b_id = self.g_ab(img_b)
        
        self.d_a.trainable = False
        self.d_b.trainable = False
        
        valid_a = self.d_a(fake_a)
        valid_b = self.d_b(fake_b)
        
        self.combined = Model(inputs=[img_a, img_b], output=[valid_a, valid_b, recon_a, recon_b, img_a_id, img_b_id])
        self.combined.compile(loss=['mse', 'mse', 'mse','mse','mae','mae'],
                             loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle, self.lambda_id, self.lambda_id], optimizer=optimizer)
        
    def plot_graph(self,model=0):
        if model == 'discriminator':
            return plot_model(self.d_a, show_shapes = True, show_layer_names = True,to_file='d.png')
        elif model == 'generator':
            return plot_model(self.g_ab, show_shapes = True, show_layer_names = True,to_file='g.png')
        else:
            return plot_model(self.combined, show_shapes = True, show_layer_names = True,to_file='c.png')
        
    @staticmethod
    def conv2d(layer_input, filters, f_size=4, normalization=True):
        """Layers used during downsampling"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if normalization:
            d = BatchNormalization()(d)

        return d

    @staticmethod
    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0.3):
        """Layers used during upsampling"""
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        u = BatchNormalization()(u)
        u = Concatenate()([u, skip_input])

        return u
    
    @staticmethod
    def residual_block(layer_input):
        """Residual block described in paper"""
        d = Conv2D(64, kernel_size=3, strides=1, padding='same')(layer_input)
        d = BatchNormalization(momentum=0.8)(d)
        d = Activation('relu')(d)
        d = Conv2D(64, kernel_size=3, strides=1, padding='same')(d)
        d = BatchNormalization(momentum=0.8)(d)
        d = Concatenate()([d, layer_input])
        return d        

    def build_generator(self,name):
        """U-net Generator"""
        d0 = Input(shape=self.img_shape)
        
        d1 = self.conv2d(d0, self.gf)
        d2 = self.conv2d(d1, self.gf * 2)
        d3 = self.conv2d(d2, self.gf * 4)
        d4 = self.conv2d(d3, self.gf * 8)
        
        u1 = self.deconv2d(d4, d3, self.gf * 4)
        u2 = self.deconv2d(u1, d2, self.gf * 2)
        u3 = self.deconv2d(u2, d1, self.gf)
        
        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)
        
        return Model(d0, output_img,name=name)
    def build_generator_resnet(self,name):
        """Resnet Generator"""

        img = Input(shape=self.img_shape)

        l1 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(img)

        r = self.residual_block(l1)
        for _ in range(self.residual_blocks - 1):
            r = self.residual_block(r)

        output_img = Conv2D(self.channels, kernel_size=3, padding='same', activation='tanh')(r)

        return Model(img, output_img,name=name)    
        
    def build_discriminator(self,name):
        img = Input(shape=self.img_shape)
        
        d1 = self.conv2d(img, self.df, normalization=False)
        d2 = self.conv2d(d1, self.df * 2)
        d3 = self.conv2d(d2, self.df * 4)
        d4 = self.conv2d(d3, self.df * 8)
        
        validity = Conv2D(1, kernel_size=4, strides=1, padding='same',activation='tanh')(d4)
        
        return Model(img, validity,name=name)
    
    def train(self, epochs, batch_size=1,save=False):
        
        g_loss_history = []
        d_loss_history = []
        
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        if 1==1:
            for epoch in tqdm(range(1, epochs + 1)):
                
                index = np.random.randint(0, x_t.shape[0], batch_size)
                imgs_a = x_i[index]
                imgs_b = x_t[index]
                fake_b = self.g_ab.predict(imgs_a)
                fake_a = self.g_ba.predict(imgs_b)
                
                da_loss_real = self.d_a.train_on_batch(imgs_a, valid)
                da_loss_fake = self.d_a.train_on_batch(fake_a, fake)
                da_loss = 0.5 * np.add(da_loss_real, da_loss_fake)

                db_loss_real = self.d_b.train_on_batch(imgs_b, valid)
                db_loss_fake = self.d_b.train_on_batch(fake_b, fake)
                db_loss = 0.5 * np.add(db_loss_real, db_loss_fake)

                d_loss = 0.5 * np.add(da_loss, db_loss)
                
                g_loss = self.combined.train_on_batch([imgs_a, imgs_b], [valid, valid, imgs_a, imgs_b, imgs_a, imgs_b])
                
                g_loss_history.append(g_loss)
                d_loss_history.append(d_loss)
                if epoch % 300 == 0:
                    index = np.random.randint(0,876, batch_size)
                    gen_AB=gan.g_ab.predict(x_i[index])
                    gen_BA=gan.g_ba.predict(x_i[index])
                    fig, axes = plt.subplots(nrows = 8, ncols = 4, figsize = (16, 16))
                    plt.setp(axes.flat, xticks = [], yticks = [])
                    for i, ax in enumerate(axes.flat):
                        #ax.imshow(gen_BA[i])
                        ax.imshow(gen_AB[i])
                    plt.show()
                    
        plt.figure(figsize = (20, 8))
        plt.plot(d_loss_history)
        plt.title('Discriminator Loss History')
        plt.show()
        if save:
            model_json = self.combined.to_json()
            with open("gan.json", "w") as json_file:
                json_file.write(model_json)

            #model.save_weights("model.h5")
            print("Model saved")

In [ ]:
gan = CycleGAN()

In [ ]:
gan.train(epochs=5000, batch_size=32,save=True)


In [ ]:
dot = gan.plot_graph('generator')
dot = gan.plot_graph('discriminator')
dot = gan.plot_graph()
dot

In [ ]:
index = np.random.randint(0,876, 16)
gen_AB=gan.g_ab.predict(test[index])
gen_BA=gan.g_ba.predict(test[index])
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (16, 16))
plt.setp(axes.flat, xticks = [], yticks = [])
for i, ax in enumerate(axes.flat):
    #ax.imshow(gen_BA[i])
    ax.imshow(cv2.cvtColor(gen_AB[i].astype('float32'), cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (16, 16))
plt.setp(axes.flat, xticks = [], yticks = [])
for i, ax in enumerate(axes.flat):
    ax.imshow(cv2.cvtColor(test[index][i].astype('float32'), cv2.COLOR_BGR2RGB))
    #ax.imshow(gen_BA[i])
plt.show()

In [ ]:

    def conv2d(layer_input, filters, f_size=4, normalization=True):
        """Layers used during downsampling"""
        with tf.variable_scope("conv2", reuse=tf.AUTO_REUSE):
            def spectral_norm(w, iteration=1):
                w_shape = w.shape.as_list()
                w = tf.reshape(w, [-1, w_shape[-1]])

                u = tf.get_variable("u", [1, w_shape[-1]], initializer=tf.random_normal_initializer(), trainable=False)

                u_hat = u
                v_hat = None
                for i in range(iteration):
                    v_ = tf.matmul(u_hat, tf.transpose(w))
                    v_hat = tf.nn.l2_normalize(v_)

                    u_ = tf.matmul(v_hat, w)
                    u_hat = tf.nn.l2_normalize(u_)

                u_hat = tf.stop_gradient(u_hat)
                v_hat = tf.stop_gradient(v_hat)

                sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

                with tf.control_dependencies([u.assign(u_hat)]):
                    w_norm = w / sigma
                    w_norm = tf.reshape(w_norm, w_shape)
                return w_norm
            
            
            w = tf.get_variable("kernel", shape=[4, 4, layer_input.get_shape()[-1], 3])
            b = tf.get_variable("bias", [3], initializer=tf.constant_initializer(0.0))
        
            x = tf.nn.bias_add(tf.nn.conv2d(input=layer_input, filter=spectral_norm(w), strides=[1, 2, 2, 1],padding='SAME'),b)
            
            d = LeakyReLU(alpha=0.2)(x)
            
            d = BatchNormalization()(d)

            return d